In [1]:
import numpy as np
import scipy.io
import networkx as nx
import pandas as pd
from matplotlib import pyplot as plt
import json

### Build Connectome DiGraph

In [2]:
def get_connectome(filepath, sheet_name):
    excel_connectome = pd.read_excel(filepath, sheet_name)
    table = excel_connectome.values[1:,2:]

    neuron_names = set().union(table[1:,0], table[0,1:])
    neuron_names = {x for x in neuron_names if x == x} #remove nans
    neuron_names = sorted(list(neuron_names))

    edge_list = []

    def isnan(x):
        return isinstance(x, float) and np.isnan(x)

    for r in range(1,len(table)):
        for c in range(1,len(table[r])):
            if isnan(table[r][c]) or isnan(table[r][0]) or isnan(table[0][c]):
                continue
            edge_list.append((table[r][0], table[0][c], table[r][c]))

    connectome = nx.DiGraph()
    connectome.add_weighted_edges_from(edge_list)  # note that nodes with no edges will not be present in graph
    return connectome

# example:
# get_connectome("datasets/NIHMS1054987-supplement-Supplementary_files/Supplementary Information/SI 5 Connectome adjacency matrices.xlsx", sheet_name="male chemical")

In [26]:
CONNECTOME = get_connectome("datasets/NIHMS1054987-supplement-Supplementary_files/Supplementary Information/SI 5 Connectome adjacency matrices.xlsx", sheet_name="male chemical")

In [24]:
fake_connectome = get_connectome('datasets/fake_test_data/fake_connectome.xlsx', 'S1S2N1')

In [28]:
def load_trace_json(json_filepath):
    with open(json_filepath) as trace_file:
        return json.load(trace_file)

# TODO: (if more than one format for trace data arises) create trace object and functions to load data from files into object.

# def get_traces(json_filepath):
#     with open(json_filepath) as trace_file:
#         trace_json = json.load(trace_file)

#     traces = np.array(trace_json['trace_array'])
#     trace_indices = {}
#     for i in range(trace_json['num_neurons']):
#         if str(i+1) in trace_json['labeled']:
#             trace_indices[trace_json['labeled'][str(i+1)]['label']] = i  # neurons are 1-indexed
    
#     return traces, trace_indices

# example:
# load_trace_json("datasets/wormwideweb/2022-08-02-01.json")

In [38]:
def get_relevant_mask(neuron, trace_json, connectome=CONNECTOME):
    relevant_neurons = set().union([neuron], connectome.successors(neuron), connectome.predecessors(neuron))
    return np.array([str(i+1) in trace_json['labeled'] and trace_json['labeled'][str(i+1)]['label'] in relevant_neurons for i in range(trace_json['num_neurons'])])

In [41]:
trace_json = load_trace_json('datasets/fake_test_data/fake_traces.json')
relevant_mask = get_relevant_mask('S2', trace_json, fake_connectome)
traces = np.array(trace_json['trace_array'])
print(traces[relevant_mask][:,:10])

[[ 6.63122658e-01  2.39315664e-01 -2.39315664e-01 -6.63122658e-01
  -9.35016243e-01 -9.92708874e-01 -8.22983866e-01 -4.64723172e-01
   6.43249060e-16  4.64723172e-01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00]]
